In [16]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from datetime import datetime
import json
import os
import folium
import warnings
warnings.filterwarnings('ignore')

from geopy.geocoders import Nominatim

# Carnevale

In [2]:
dr_notebook = '/Users/giuliarovinelli/Desktop/Università/PhD/actv/notebookMatteo/transfer_files/csv/'
dataset_compl = pd.read_csv(dr_notebook+'dataset_cleaned_tempesportazioneCompleta.csv')
print(f'dataset_compl.shape: {dataset_compl.shape}')
dataset_compl.head()

dataset_compl.shape: (4947477, 9)


,DATA,ORA,DATA_VALIDAZIONE,FERMATA,DESCRIZIONE,TICKET_CODE,DESCRIZIONE_TITOLO,LATITUDE,LONGITUDE
0,2023-01-13,00:00:00,2023-01-13 00:00:00,162,Stazione MES,7,BIGL.AUT.75'MESTRE/LIDO-TSC,45.482675,12.231809
1,2023-01-13,05:21:00,2023-01-13 05:21:00,162,Stazione MES,5,MENSILE ORDINARIO RETE UNICA,45.482675,12.231809
2,2023-01-13,07:01:00,2023-01-13 07:01:00,162,Stazione MES,5,MENSILE ORDINARIO RETE UNICA,45.482675,12.231809
3,2023-01-13,07:02:00,2023-01-13 07:02:00,162,Stazione MES,5,MENSILE ORDINARIO RETE UNICA,45.482675,12.231809
4,2023-01-13,07:36:00,2023-01-13 07:36:00,162,Stazione MES,7,BIGL.AUT.75'MESTRE/LIDO-TSC,45.482675,12.231809


In [3]:
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.491853) & 
                  (dataset_compl["LONGITUDE"] == 12.242548), "FERMATA"] = -1
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.438038) & 
                  (dataset_compl["LONGITUDE"] == 12.318223), "FERMATA"] = -2
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.482675) & 
                  (dataset_compl["LONGITUDE"] == 12.231809), "FERMATA"] = -3
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.504976) & 
                  (dataset_compl["LONGITUDE"] == 12.339106), "FERMATA"] = -4
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.390778) & 
                  (dataset_compl["LONGITUDE"] == 12.353001), "FERMATA"] = -5


dataset_compl.loc[(dataset_compl["FERMATA"] == -1), "DESCRIZIONE"] = "TERRA"
dataset_compl.loc[(dataset_compl["FERMATA"] == -2), "DESCRIZIONE"] = "Piazzale Roma"
dataset_compl.loc[(dataset_compl["FERMATA"] == -3), "DESCRIZIONE"] = "Stazione Mestre"
dataset_compl.loc[(dataset_compl["FERMATA"] == -4), "DESCRIZIONE"] = "Aeroporto"
dataset_compl.loc[(dataset_compl["FERMATA"] == -5), "DESCRIZIONE"] = "Lido bus"

dataset_compl.loc[(dataset_compl["FERMATA"] == 15076), "FERMATA"] = 5076
dataset_compl.loc[(dataset_compl["FERMATA"] == 15065), "FERMATA"] = 5065
dataset_compl.loc[(dataset_compl["FERMATA"] == 15064), "FERMATA"] = 5064
dataset_compl.loc[(dataset_compl["FERMATA"] == 15011), "FERMATA"] = 5011
dataset_compl.loc[(dataset_compl["FERMATA"] == 15066), "FERMATA"] = 5066
dataset_compl.loc[(dataset_compl["FERMATA"] == 15117), "FERMATA"] = 5117
dataset_compl.loc[(dataset_compl["FERMATA"] == 5618), "FERMATA"] = 5018

dataset_compl.loc[(dataset_compl["FERMATA"] == 5076), "DESCRIZIONE"] = "San Marco-Sa"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5065), "DESCRIZIONE"] = "Murano Museo"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5064), "DESCRIZIONE"] = "Murano Navag"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5011), "DESCRIZIONE"] = "San Marco-Sa"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5066), "DESCRIZIONE"] = "Murano Venie"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5117), "DESCRIZIONE"] = "Murano Da Mu"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5018), "DESCRIZIONE"] = "Certosa"

dataset_compl.loc[(dataset_compl["FERMATA"] == 5049), "DESCRIZIONE"] = "Zattere \"B\""
dataset_compl.loc[(dataset_compl["FERMATA"] == 5009), "DESCRIZIONE"] = "San Marco-Sa"

In [4]:
dataset_compl[dataset_compl['FERMATA']==5009]

print(dataset_compl.dtypes)

DATA                   object
ORA                    object
DATA_VALIDAZIONE       object
FERMATA                 int64
DESCRIZIONE            object
TICKET_CODE            object
DESCRIZIONE_TITOLO     object
LATITUDE              float64
LONGITUDE             float64
dtype: object


In [5]:
num_fermate = dataset_compl['FERMATA'].nunique()
print(f'Numero di fermate distinte: {num_fermate}')

Numero di fermate distinte: 126


In [6]:
#dataset_compl_isola_fermate = dataset_compl.copy()

#dataset_compl_isola_fermate = dataset_compl_isola_fermate[()]

dataset_compl['DATA_VALIDAZIONE'] = pd.to_datetime(dataset_compl['DATA_VALIDAZIONE'])
dataset_compl['DATA'] = pd.to_datetime(dataset_compl['DATA'])
#dataset_compl.dtypes

# Print the interval of dates for which we have data
print(f'Date range: {dataset_compl.DATA_VALIDAZIONE.min()} to {dataset_compl.DATA_VALIDAZIONE.max()}')

Date range: 2023-01-13 00:00:00 to 2023-03-14 23:59:00


In [7]:
dataset_clust = dataset_compl.copy()
#dataset_clust.drop(columns=['DATA', 'ORA','DESCRIZIONE_TITOLO'],inplace=True)
dataset_clust.drop(columns=['DESCRIZIONE_TITOLO'],inplace=True)
dataset_clust.head(3)

,DATA,ORA,DATA_VALIDAZIONE,FERMATA,DESCRIZIONE,TICKET_CODE,LATITUDE,LONGITUDE
0,2023-01-13,00:00:00,2023-01-13 00:00:00,-3,Stazione Mestre,7,45.482675,12.231809
1,2023-01-13,05:21:00,2023-01-13 05:21:00,-3,Stazione Mestre,5,45.482675,12.231809
2,2023-01-13,07:01:00,2023-01-13 07:01:00,-3,Stazione Mestre,5,45.482675,12.231809


In [8]:
lat_lon = dataset_compl.copy()
lat_lon = lat_lon[['LATITUDE', 'LONGITUDE','FERMATA']].drop_duplicates()
lat_lon.reset_index(drop=True,inplace=True)
lat_lon

,LATITUDE,LONGITUDE,FERMATA
0,45.482675,12.231809,-3
1,45.438038,12.318223,-2
2,45.491853,12.242548,-1
3,45.504976,12.339106,-4
4,45.390778,12.353001,-5
...,...,...,...
121,45.432186,12.337525,5054
122,45.429008,12.315648,5023
123,45.444424,12.324183,5189
124,45.454956,12.386839,5610


In [9]:
duplicates = lat_lon.duplicated(subset=['LATITUDE', 'LONGITUDE'])
duplicate_rows = lat_lon[duplicates]
duplicate_rows

,LATITUDE,LONGITUDE,FERMATA


In [10]:
fermate = dataset_compl.copy()
fermate = fermate[['LATITUDE', 'LONGITUDE','FERMATA','DESCRIZIONE']].drop_duplicates()
fermate.reset_index(drop=True,inplace=True)
print(f'fermate.shape: {fermate.shape}')
fermate.head()

fermate.shape: (126, 4)


,LATITUDE,LONGITUDE,FERMATA,DESCRIZIONE
0,45.482675,12.231809,-3,Stazione Mestre
1,45.438038,12.318223,-2,Piazzale Roma
2,45.491853,12.242548,-1,TERRA
3,45.504976,12.339106,-4,Aeroporto
4,45.390778,12.353001,-5,Lido bus


In [13]:
num_fermate = dataset_compl['FERMATA'].nunique()
print(f'Numero di fermate distinte: {num_fermate}')

Numero di fermate distinte: 126


In [12]:
print(dataset_clust.TICKET_CODE.unique())

['7' '5' '6-STUD' '4' '5-STUD' '3' '6-RET' '6' '2' '1' '5-WKRS' '6-WKRS']


In [15]:
dataset_compl.to_csv(dr_notebook+'csvPuliti/'+'dataset_cleaned_tempesportazioneCompleta.csv',index=False)

# Estate 2022

In [33]:
dr_notebook = '/Users/giuliarovinelli/Desktop/Università/PhD/actv/notebookMatteo/transfer_files/csv/'
dataset_compl = pd.read_csv(dr_notebook+'dataset_cleaned_tempvalidazioni.csv')
print(f'dataset_compl.shape: {dataset_compl.shape}')
dataset_compl.head()

dataset_compl.shape: (4427619, 9)


,DATA,ORA,DATA_VALIDAZIONE,FERMATA,DESCRIZIONE,TICKET_CODE,DESCRIZIONE_TITOLO,LATITUDE,LONGITUDE
0,2022-05-13,00:00:00,2022-05-13 00:00:00,506,VENEZIA,1,DAILY PASS VENEZIA - AVM,45.438038,12.318223
1,2022-05-13,00:00:00,2022-05-13 00:00:00,506,VENEZIA,1,DAILY PASS VENEZIA - AVM,45.438038,12.318223
2,2022-05-13,00:00:00,2022-05-13 00:00:00,506,VENEZIA,1,DAILY PASS VENEZIA - AVM,45.438038,12.318223
3,2022-05-13,00:00:00,2022-05-13 00:00:00,506,VENEZIA,1,DAILY PASS VENEZIA - AVM,45.438038,12.318223
4,2022-05-13,00:01:00,2022-05-13 00:01:00,506,VENEZIA,3,72H R.VENICE+AEROP.AR ONLINE,45.438038,12.318223


In [34]:
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.491853) & 
                  (dataset_compl["LONGITUDE"] == 12.242548), "FERMATA"] = -1
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.438038) & 
                  (dataset_compl["LONGITUDE"] == 12.318223), "FERMATA"] = -2
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.482675) & 
                  (dataset_compl["LONGITUDE"] == 12.231809), "FERMATA"] = -3
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.504976) & 
                  (dataset_compl["LONGITUDE"] == 12.339106), "FERMATA"] = -4
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.390778) & 
                  (dataset_compl["LONGITUDE"] == 12.353001), "FERMATA"] = -5


dataset_compl.loc[(dataset_compl["FERMATA"] == -1), "DESCRIZIONE"] = "TERRA"
dataset_compl.loc[(dataset_compl["FERMATA"] == -2), "DESCRIZIONE"] = "Piazzale Roma"
dataset_compl.loc[(dataset_compl["FERMATA"] == -3), "DESCRIZIONE"] = "Stazione Mestre"
dataset_compl.loc[(dataset_compl["FERMATA"] == -4), "DESCRIZIONE"] = "Aeroporto"
dataset_compl.loc[(dataset_compl["FERMATA"] == -5), "DESCRIZIONE"] = "Lido bus"

dataset_compl.loc[(dataset_compl["FERMATA"] == 15076), "FERMATA"] = 5076
dataset_compl.loc[(dataset_compl["FERMATA"] == 15065), "FERMATA"] = 5065
dataset_compl.loc[(dataset_compl["FERMATA"] == 15066), "FERMATA"] = 5066
dataset_compl.loc[(dataset_compl["FERMATA"] == 5618), "FERMATA"] = 5018
dataset_compl.loc[(dataset_compl["FERMATA"] == 15117), "FERMATA"] = 5117
dataset_compl.loc[(dataset_compl["FERMATA"] == 15064), "FERMATA"] = 5064
dataset_compl.loc[(dataset_compl["FERMATA"] == 15011), "FERMATA"] = 5011

dataset_compl.loc[(dataset_compl["FERMATA"] == 5076), "DESCRIZIONE"] = "San Marco-Sa"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5065), "DESCRIZIONE"] = "Murano Museo"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5066), "DESCRIZIONE"] = "Murano Venie"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5018), "DESCRIZIONE"] = "Certosa"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5117), "DESCRIZIONE"] = "Murano Da Mu"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5064), "DESCRIZIONE"] = "Murano Navag"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5011), "DESCRIZIONE"] = "San Marco-Sa"

#dataset_compl.loc[(dataset_compl["FERMATA"] == 5049), "DESCRIZIONE"] = "Zattere \"B\""
#dataset_compl.loc[(dataset_compl["FERMATA"] == 5009), "DESCRIZIONE"] = "San Marco-Sa"

In [35]:
num_fermate = dataset_compl['FERMATA'].nunique()
print(f'Numero di fermate distinte: {num_fermate}')

Numero di fermate distinte: 127


In [36]:
lat_lon = dataset_compl.copy()
lat_lon = lat_lon[['LATITUDE', 'LONGITUDE','FERMATA']].drop_duplicates()
lat_lon.reset_index(drop=True,inplace=True)
lat_lon

,LATITUDE,LONGITUDE,FERMATA
0,45.438038,12.318223,-2
1,45.491853,12.242548,-1
2,45.504976,12.339106,-4
3,45.390778,12.353001,-5
4,45.482675,12.231809,-3
...,...,...,...
122,45.426624,12.337678,5118
123,45.432774,12.348552,5008
124,45.429760,12.354997,5124
125,45.446976,12.424375,5109


In [37]:
duplicates = lat_lon.duplicated(subset=['LATITUDE', 'LONGITUDE'])
duplicate_rows = lat_lon[duplicates]
duplicate_rows

,LATITUDE,LONGITUDE,FERMATA


In [42]:
#dataset_compl[(dataset_compl['FERMATA']==5009)]

In [43]:
#dataset_compl['DESCRIZIONE'].unique()

In [44]:
dataset_compl.to_csv(dr_notebook+'csvPuliti/'+'dataset_cleaned_tempvalidazioni.csv',index=False)

# Primavera 2023

In [50]:
dr_notebook = '/Users/giuliarovinelli/Desktop/Università/PhD/actv/notebookMatteo/transfer_files/csv/'
dataset_compl = pd.read_csv(dr_notebook+'dataset_cleaned_tempesportazionePasqua23_part1.csv')
print(f'dataset_compl.shape: {dataset_compl.shape}')
dataset_compl.head()

dataset_compl.shape: (3847430, 9)


,DATA,ORA,DATA_VALIDAZIONE,FERMATA,DESCRIZIONE,TICKET_CODE,DESCRIZIONE_TITOLO,LATITUDE,LONGITUDE
0,2023-04-04,00:01:00,2023-04-04 00:01:00,3625,Aeroporto MA,7,BIGL RETE UNICA 75',45.504976,12.339106
1,2023-04-04,00:02:00,2023-04-04 00:02:00,3625,Aeroporto MA,2,48H ONLINE AEROBUS CS 1 MESE,45.504976,12.339106
2,2023-04-04,00:02:00,2023-04-04 00:02:00,3625,Aeroporto MA,2,48H ONLINE AEROBUS CS 1 MESE,45.504976,12.339106
3,2023-04-04,00:05:00,2023-04-04 00:05:00,3625,Aeroporto MA,7,BIGL RETE UNICA 75',45.504976,12.339106
4,2023-04-04,00:05:00,2023-04-04 00:05:00,3625,Aeroporto MA,7,BIGL RETE UNICA 75',45.504976,12.339106


In [51]:
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.491853) & 
                  (dataset_compl["LONGITUDE"] == 12.242548), "FERMATA"] = -1
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.438038) & 
                  (dataset_compl["LONGITUDE"] == 12.318223), "FERMATA"] = -2
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.482675) & 
                  (dataset_compl["LONGITUDE"] == 12.231809), "FERMATA"] = -3
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.504976) & 
                  (dataset_compl["LONGITUDE"] == 12.339106), "FERMATA"] = -4
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.390778) & 
                  (dataset_compl["LONGITUDE"] == 12.353001), "FERMATA"] = -5

dataset_compl.loc[(dataset_compl["FERMATA"] == -1), "DESCRIZIONE"] = "TERRA"
dataset_compl.loc[(dataset_compl["FERMATA"] == -2), "DESCRIZIONE"] = "Piazzale Roma"
dataset_compl.loc[(dataset_compl["FERMATA"] == -3), "DESCRIZIONE"] = "Stazione Mestre"
dataset_compl.loc[(dataset_compl["FERMATA"] == -4), "DESCRIZIONE"] = "Aeroporto"
dataset_compl.loc[(dataset_compl["FERMATA"] == -5), "DESCRIZIONE"] = "Lido bus"


dataset_compl.loc[(dataset_compl["FERMATA"] == 15076), "FERMATA"] = 5076
dataset_compl.loc[(dataset_compl["FERMATA"] == 15065), "FERMATA"] = 5065
dataset_compl.loc[(dataset_compl["FERMATA"] == 15066), "FERMATA"] = 5066
dataset_compl.loc[(dataset_compl["FERMATA"] == 5618), "FERMATA"] = 5018
dataset_compl.loc[(dataset_compl["FERMATA"] == 15117), "FERMATA"] = 5117
dataset_compl.loc[(dataset_compl["FERMATA"] == 15064), "FERMATA"] = 5064
#dataset_compl.loc[(dataset_compl["FERMATA"] == 15011), "FERMATA"] = 5011

dataset_compl.loc[(dataset_compl["FERMATA"] == 5076), "DESCRIZIONE"] = "San Marco-Sa"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5065), "DESCRIZIONE"] = "Murano Museo"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5066), "DESCRIZIONE"] = "Murano Venie"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5018), "DESCRIZIONE"] = "Certosa"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5117), "DESCRIZIONE"] = "Murano Da Mu"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5064), "DESCRIZIONE"] = "Murano Navag"
#dataset_compl.loc[(dataset_compl["FERMATA"] == 5011), "DESCRIZIONE"] = "San Marco-Sa"

In [52]:
num_fermate = dataset_compl['FERMATA'].nunique()
print(f'Numero di fermate distinte: {num_fermate}')

Numero di fermate distinte: 118


In [53]:
lat_lon = dataset_compl.copy()
lat_lon = lat_lon[['LATITUDE', 'LONGITUDE','FERMATA']].drop_duplicates()
lat_lon.reset_index(drop=True,inplace=True)
lat_lon

,LATITUDE,LONGITUDE,FERMATA
0,45.504976,12.339106,-4
1,45.438038,12.318223,-2
2,45.491853,12.242548,-1
3,45.390778,12.353001,-5
4,45.482675,12.231809,-3
...,...,...,...
113,45.432446,12.338422,5056
114,45.444424,12.324183,5189
115,45.441830,12.325079,5136
116,45.440308,12.306533,5025


In [54]:
duplicates = lat_lon.duplicated(subset=['LATITUDE', 'LONGITUDE'])
duplicate_rows = lat_lon[duplicates]
duplicate_rows

,LATITUDE,LONGITUDE,FERMATA


In [56]:
dataset_compl.to_csv(dr_notebook+'csvPuliti/'+'dataset_cleaned_tempesportazionePasqua23_part1.csv',index=False)

# Fine primavera 2023

In [62]:
dr_notebook = '/Users/giuliarovinelli/Desktop/Università/PhD/actv/notebookMatteo/transfer_files/csv/'
dataset_compl = pd.read_csv(dr_notebook+'dataset_cleaned_tempesportazionePasqua23_part2.csv')
print(f'dataset_compl.shape: {dataset_compl.shape}')
dataset_compl.head()

dataset_compl.shape: (3893477, 9)


,DATA,ORA,DATA_VALIDAZIONE,FERMATA,DESCRIZIONE,TICKET_CODE,DESCRIZIONE_TITOLO,LATITUDE,LONGITUDE
0,2023-05-04,00:00:00,2023-05-04 00:00:00,4641,Santa Maria,4,"7GG-TPL 48,60-COMVE16,40",45.390778,12.353001
1,2023-05-04,00:00:00,2023-05-04 00:00:00,4641,Santa Maria,4,"7GG-TPL 48,60-COMVE16,40",45.390778,12.353001
2,2023-05-04,07:06:00,2023-05-04 07:06:00,4641,Santa Maria,7,BIGL RETE UNICA 75',45.390778,12.353001
3,2023-05-04,07:24:00,2023-05-04 07:24:00,4641,Santa Maria,6,ANNUALE ORDINARIO RETE UNICA,45.390778,12.353001
4,2023-05-04,07:25:00,2023-05-04 07:25:00,4641,Santa Maria,7,BIGL RETE UNICA 75',45.390778,12.353001


In [63]:
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.491853) & 
                  (dataset_compl["LONGITUDE"] == 12.242548), "FERMATA"] = -1
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.438038) & 
                  (dataset_compl["LONGITUDE"] == 12.318223), "FERMATA"] = -2
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.482675) & 
                  (dataset_compl["LONGITUDE"] == 12.231809), "FERMATA"] = -3
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.504976) & 
                  (dataset_compl["LONGITUDE"] == 12.339106), "FERMATA"] = -4
dataset_compl.loc[(dataset_compl["LATITUDE"] == 45.390778) & 
                  (dataset_compl["LONGITUDE"] == 12.353001), "FERMATA"] = -5

dataset_compl.loc[(dataset_compl["FERMATA"] == -1), "DESCRIZIONE"] = "TERRA"
dataset_compl.loc[(dataset_compl["FERMATA"] == -2), "DESCRIZIONE"] = "Piazzale Roma"
dataset_compl.loc[(dataset_compl["FERMATA"] == -3), "DESCRIZIONE"] = "Stazione Mestre"
dataset_compl.loc[(dataset_compl["FERMATA"] == -4), "DESCRIZIONE"] = "Aeroporto"
dataset_compl.loc[(dataset_compl["FERMATA"] == -5), "DESCRIZIONE"] = "Lido bus"


dataset_compl.loc[(dataset_compl["FERMATA"] == 15076), "FERMATA"] = 5076
dataset_compl.loc[(dataset_compl["FERMATA"] == 15065), "FERMATA"] = 5065
dataset_compl.loc[(dataset_compl["FERMATA"] == 5618), "FERMATA"] = 5018
dataset_compl.loc[(dataset_compl["FERMATA"] == 15066), "FERMATA"] = 5066
dataset_compl.loc[(dataset_compl["FERMATA"] == 15117), "FERMATA"] = 5117
dataset_compl.loc[(dataset_compl["FERMATA"] == 15064), "FERMATA"] = 5064
#dataset_compl.loc[(dataset_compl["FERMATA"] == 15011), "FERMATA"] = 5011

dataset_compl.loc[(dataset_compl["FERMATA"] == 5076), "DESCRIZIONE"] = "San Marco-Sa"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5065), "DESCRIZIONE"] = "Murano Museo"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5066), "DESCRIZIONE"] = "Murano Venie"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5018), "DESCRIZIONE"] = "Certosa"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5117), "DESCRIZIONE"] = "Murano Da Mu"
dataset_compl.loc[(dataset_compl["FERMATA"] == 5064), "DESCRIZIONE"] = "Murano Navag"
#dataset_compl.loc[(dataset_compl["FERMATA"] == 5011), "DESCRIZIONE"] = "San Marco-Sa"

In [64]:
num_fermate = dataset_compl['FERMATA'].nunique()
print(f'Numero di fermate distinte: {num_fermate}')

Numero di fermate distinte: 120


In [65]:
lat_lon = dataset_compl.copy()
lat_lon = lat_lon[['LATITUDE', 'LONGITUDE','FERMATA']].drop_duplicates()
lat_lon.reset_index(drop=True,inplace=True)
lat_lon

,LATITUDE,LONGITUDE,FERMATA
0,45.390778,12.353001,-5
1,45.482675,12.231809,-3
2,45.491853,12.242548,-1
3,45.438038,12.318223,-2
4,45.504976,12.339106,-4
...,...,...,...
115,45.432774,12.348552,5008
116,45.429008,12.315648,5023
117,45.243614,12.294001,5017
118,45.426899,12.325401,5129


In [66]:
duplicates = lat_lon.duplicated(subset=['LATITUDE', 'LONGITUDE'])
duplicate_rows = lat_lon[duplicates]
duplicate_rows

,LATITUDE,LONGITUDE,FERMATA


In [67]:
dataset_compl.to_csv(dr_notebook+'csvPuliti/'+'dataset_cleaned_tempesportazionePasqua23_part2.csv',index=False)